(application)=
# Application runtime

You can use the {py:meth}`~mlrun.runtimes.ApplicationRuntime` to provide an image that runs on top of your deployed model. 

The application runtime deploys a container image (for example, a web application) that is exposed on a specific port, and a command to run the HTTP server. The runtime is based on top of Nuclio, and adds the application as a side-car to a Nuclio function pod while the actual function is a reverse proxy to that application. 

You can set an existing image to run in the application, or let the application runtime build the side-car image for you, by specifying the source code, and pulling at build-time. 

> Note: The default base image is `python:3.9` when not specifying and image for the application.
    
An API Gateway, by default, is in front of the application and can provide different authentication methods, or none.

Typical use cases are:
- Deploy a [Vizro](https://github.com/mckinsey/vizro) dashboard that communicates with an external source (for example, a serving model) to display graphs, data, and inference.
- Deploy a model and a UI &mdash; the model serving is the backend and the UI is the side car.
- Deploy a fastapi web-server with an MLRun model. In this case, the Nuclio function is a reverse proxy and the user web-app is the side car.


## Usage examples

Deploy a Vizro dashboard from a pre-built image:

In [ ]:
# Create an application runtime (with pre-built image)
application = project.set_function(
    name="my-vizro-dashboard", kind="application", image="repo/my-vizro-image:latest"
)
# Set the port that the side-car listens on
application.set_internal_application_port(port=8050)

# Deploy
application.deploy()

Deploy a Vizro dashboard from a source archive or git:

In [ ]:
# Specify the source to be loaded at build-time or run-time
application = project.set_function(
    name="my-vizro-dashboard", kind="application", requirements=["vizro"]
)
application.set_internal_application_port(8050)
application.spec.command = "gunicorn"
application.spec.args = [
    "<my-app>:<my-server>",
    "--bind",
    "0.0.0.0:8050",
]

# Provide code artifacts
application.with_source_archive(
    "git://github.com/org/repo#my-branch", pull_at_runtime=False
)

# Build the application image via MLRun and deploy the Nuclio function
# Optionally add mlrun
application.deploy(with_mlrun=False)

Reusing an already built reverse proxy image is done when:
- Redeploying a function that built the reverse proxy once and has `application.status.container_image` enriched.
- It was already built manually with `mlrun.runtimes.ApplicationRuntime.deploy_reverse_proxy_image()`.

Using one of the above options can help minimize redundant builds and streamline the development cycle.

### combining application with serving function
In this example, we deploy a serving function and use it with our application.
</br></br>
<u>Serving creation</u>:</br>
First, we will create our model file and save it as a .py file.

In [ ]:
# demo, saved in /your/path/add_ten_model.py
from mlrun.serving import V2ModelServer

class AddTenModel(V2ModelServer):
    def load(self):
        # No actual model to load, just a demo
        pass

    def predict(self, request):
        input = request['inputs'][0]
        result = input + 10
        return {"outputs": [result]}

Now we will deploy this model as a serving function:

In [ ]:
# first we will create a demo model, using V2ModelServer, we will save this file as .py file.

# Import MLRun and create the serving function
import mlrun
project_name = "serving-project"
model_name = "add-ten-model"
model_path = "/your/path/add_ten_model.py"

project = mlrun.get_or_create_project(project_name)

In [ ]:
# Create the serving function
function = mlrun.code_to_function(name='add-ten-serving',
                                  kind='serving',
                                  image='your-image', # should include mlrun for importing V2ModelServer
                                  filename=model_path)

In [ ]:
# Add the model to the function (even though there's no real model in this case)
function.add_model(model_name, model_path=model_path,class_name='AddTenModel')
function.deploy()

In [ ]:
# An example of invoke:
function.invoke(f'/v2/models/{model_name}/infer', {"inputs":[20]})['outputs']['outputs']

<u>Application creation</u>:</br>
At the begining, ee will create our flask server application.</br>
The application will include several different endpoints in order to check its functionality:

In [ ]:
import os

from flask import Flask
import requests
import mlrun

app = Flask(__name__)


@app.route('/')
def hello():
    return "Hello, World!"


@app.route('/docs')
def docs():
    #  The application has access to the internet and is able to create requests.
    return {"status_code": requests.get("https://docs.mlrun.org/").status_code}


@app.route('/internal')
def internal():
    # Test access to the serving function with MLRun.
    project_name = "serving-project"
    project = mlrun.get_or_create_project(project_name)
    function = project.get_function('add-ten-serving')
    response = function.invoke('/v2/models/add-ten-model/infer', {"inputs": [20]})
    output = response['outputs']['outputs']
    return {"result": output}


@app.route('/external')
def external():
    # Test access to the serving function without MLRun (externally).
    project_name = "serving-project"
    project = mlrun.get_or_create_project(project_name)
    function = project.get_function('add-ten-serving')
    url = f"https://{function.status.external_invocation_urls[0]}"
    response = requests.post(url, json={"inputs": [50]}).json()
    output = response['outputs']['outputs']
    return {'result': output}


@app.route('/secret')
def secret():
    # Every application has access to project secrets.
    # This is just an example; do not return real secrets!
    project_secret = os.environ.get('secret-example')
    return {'secret': project_secret}


Let's save this code block as Python files and zip it into a .tar.gz file:

In [ ]:
!tar -czvf archive.tar.gz flask_app_example.py

Now, we are going to deploy this flask application:

In [ ]:
import mlrun
project = mlrun.get_or_create_project("app-demo-flask")
# Create a demo secret for testing
project.set_secrets(secrets={"secret-example":"project_secret_example"})

In [ ]:
# Specify source to be loaded on build time or run time
# The image or the requirements should include mlrun, flask and gunicorn.
application = project.set_function(name="flask_app", kind="application", image="your-image", requirements_file="/your/path/requirements.txt")

In [ ]:
# Provide code artifacts
application.with_source_archive("v3io:///your/path/archive.tar.gz", pull_at_runtime=False)

In [ ]:
application.set_internal_application_port(5000)
application.spec.command = "gunicorn"
application.spec.args = [
    "flask_app_example:app",
    "--bind",
    "127.0.0.1:5000",
    "--log-level",
    "debug",
]
application.deploy(with_mlrun=False)  # MLRun is already included in the requirements / image.

In [5]:
#  test the deployment:
application.invoke("/")

In [ ]:
application.invoke("/external").json()